In [11]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/conference-experment', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules', '/tmp/tmphhta90ja']


In [12]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [13]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [14]:
test_target_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1976
})

In [22]:
from adapters import AutoAdapterModel
from tqdm import tqdm

from transformers import AutoConfig

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [23]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack

adapter_name= "task_adapter_after_unipelt_mlm"
lang_adapter_config = AdapterConfig.load("unipelt", reduction_factor=2)
model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/unipelt_source_mlm", config=lang_adapter_config)
model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/unipelt_target_mlm", config=lang_adapter_config)
model.add_adapter(adapter_name, config="unipelt")
model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.train_adapter([adapter_name])


In [24]:
model.active_adapters = Stack("unipelt_source_mlm", adapter_name)

fn.print_trainable_parameters(model)

trainable params: 8203241 || all params: 86050905 || trainable%: 9.533009559864594


In [25]:
model.adapter_summary(as_dict=True)

[{'name': 'unipelt_source_mlm',
  'architecture': 'union',
  'active': True,
  '#param': 5742392,
  'train': False,
  '%param': 8.653018072753925},
 {'name': 'unipelt_target_mlm',
  'architecture': 'union',
  'active': False,
  '#param': 5742392,
  'train': False,
  '%param': 8.653018072753925},
 {'name': 'task_adapter_after_unipelt_mlm',
  'architecture': 'union',
  'active': True,
  '#param': 5742392,
  'train': True,
  '%param': 8.653018072753925},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [26]:
source_data

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 75013
})

In [27]:
from transformers import TrainingArguments, EvalPrediction,default_data_collator
from adapters import AdapterTrainer
eval_data = None
batch_size = 32

logging_steps = len(source_data) // batch_size

training_args = TrainingArguments(
        
        output_dir=f"{config.Config.RESULTS_SAVE_PATH}/task_after_mlm/results",                 # Where to store the output (checkpoints and predictions)
        num_train_epochs=8,                     # Total number of training epochs
        per_device_train_batch_size=batch_size,         # Batch size for training
        per_device_eval_batch_size=batch_size,          # Batch size for evaluation
        warmup_steps=500,                       # Number of warmup steps for learning rate scheduler
        learning_rate=1e-4,
        weight_decay=0.01,                      # Strength of weight decay
        logging_dir=f"{config.Config.RESULTS_SAVE_PATH}/task_after_mlm/logs",                   # Directory for storing logs
        logging_steps=logging_steps,                       # Log every X updates steps
        remove_unused_columns=False,
        evaluation_strategy="steps" if eval_data is not None else "no",            # Evaluate model every X steps
        eval_steps=logging_steps,                         # Number of steps to perform evaluation
        save_steps=logging_steps,                         # Save checkpoint every X steps
        save_total_limit=2,                     # Limit the total amount of checkpoints
        load_best_model_at_end=True if eval_data is not None else False,            # Load the best model when finished training
        report_to="none"                        # Do not report to any online service
    ) 
trainer = AdapterTrainer(
        model=model,                           # The instantiated 🤗 Transformers model to be trained
        args=training_args,                    # Training arguments, defined above
        train_dataset=source_data,           # Training dataset
        )

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [28]:
trainer.train()


  0%|          | 0/18760 [00:00<?, ?it/s]

{'loss': 0.841, 'learning_rate': 8.990142387732749e-05, 'epoch': 1.0}
{'loss': 0.6579, 'learning_rate': 7.706462212486309e-05, 'epoch': 2.0}
{'loss': 0.6087, 'learning_rate': 6.422782037239868e-05, 'epoch': 3.0}
{'loss': 0.5771, 'learning_rate': 5.1391018619934285e-05, 'epoch': 4.0}
{'loss': 0.5518, 'learning_rate': 3.855421686746988e-05, 'epoch': 5.0}
{'loss': 0.5322, 'learning_rate': 2.5717415115005478e-05, 'epoch': 6.0}
{'loss': 0.5137, 'learning_rate': 1.2880613362541074e-05, 'epoch': 7.0}
{'loss': 0.5017, 'learning_rate': 4.381161007667032e-08, 'epoch': 8.0}
{'train_runtime': 2468.2587, 'train_samples_per_second': 243.128, 'train_steps_per_second': 7.6, 'train_loss': 0.5980198588960969, 'epoch': 8.0}


TrainOutput(global_step=18760, training_loss=0.5980198588960969, metrics={'train_runtime': 2468.2587, 'train_samples_per_second': 243.128, 'train_steps_per_second': 7.6, 'train_loss': 0.5980198588960969, 'epoch': 8.0})

In [29]:
model.active_adapters = Stack("union_target_mlm", adapter_name)

model.adapter_summary(as_dict=True)

ValueError: No adapter with name 'union_target_mlm' found. Please make sure that all specified adapters are correctly loaded.

In [30]:
import numpy as np
from transformers import EvalPrediction
from sklearn.metrics import accuracy_score, f1_score,precision_recall_fscore_support


def compute_metrics(pred:EvalPrediction):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    eval_dataset=test_target_data,
    compute_metrics=compute_metrics,
)
eval_trainer.evaluate()

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/247 [00:00<?, ?it/s]

{'eval_loss': 0.7744819521903992,
 'eval_accuracy': 0.6928137651821862,
 'eval_f1': 0.695254146957065,
 'eval_precision': 0.7420792560664218,
 'eval_recall': 0.6928137651821862,
 'eval_runtime': 4.6968,
 'eval_samples_per_second': 420.711,
 'eval_steps_per_second': 52.589}

In [13]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name,with_head=True)